In [1]:
import pandas as pd

In [2]:
input_files = ["../../data/raw_data/b-h.csv", "../../data/raw_data/k-g.csv"]
output_file = "../data/1_merged_typed_data.parquet"

In [3]:
dfs = []
dfs_clean = []
for input_f in input_files:
    dfs.append(pd.read_csv(input_f,
                           engine='python',
                           on_bad_lines='skip',
                           na_values=['', '?']
    ))

In [4]:
COLUMNS_TO_DROP = ['ID', 'Name', 'Callsign', 'MMSI', 'AisSourcen']

for df in dfs:
    df_clean = df.drop(columns=COLUMNS_TO_DROP, errors='ignore')
    df_clean['StartTime'] = pd.to_datetime(df_clean['StartTime'], utc=True)
    df_clean['EndTime'] = pd.to_datetime(df_clean['EndTime'], utc=True)
    df_clean['time'] = pd.to_datetime(df_clean['time'], utc=True)  # Appears to have timezone info (+01:00)

    df_clean['StartPort'] = df_clean['StartPort'].astype('string').astype('category')
    df_clean['EndPort'] = df_clean['EndPort'].astype('string').astype('category')
    df_clean['shiptype'] = df_clean['shiptype'].astype('category')
    df_clean['Destination'] = df_clean['Destination'].astype('string').astype('category')

    dfs_clean.append(df_clean)

In [5]:
def check_mixed_types_and_examples(df):
    mixed = []
    for column in df.columns:
        unique_types = df[column].apply(type).unique()
        if len(unique_types) > 1:
            print(f"Column '{column}' has mixed types: {unique_types}")
            # Display examples of string and float values
            string_values = df[column][df[column].apply(type) == str].head()
            float_values = df[column][df[column].apply(type) == float].head()
            print(f"Examples of string values in '{column}':\n{string_values}")
            print(f"Examples of float values in '{column}':\n{float_values}")
            mixed.append(column)
    return mixed

check_mixed_types_and_examples(dfs_clean[0])

Column 'Destination' has mixed types: [<class 'str'> nan]
Examples of string values in 'Destination':
0    HAMBURG
1    HAMBURG
2    HAMBURG
3    HAMBURG
4    HAMBURG
Name: Destination, dtype: category
Categories (53, string): [BLEXEN.ROAD, BREMENHAVEN, BREMERHAVEN, BREMERHAVEN.VIA.NOK, ..., HHLO.PS, NORDENHAM, SEAHU.>.DEBRV, STADE]
Examples of float values in 'Destination':
Series([], Name: Destination, dtype: category
Categories (53, string): [BLEXEN.ROAD, BREMENHAVEN, BREMERHAVEN, BREMERHAVEN.VIA.NOK, ..., HHLO.PS, NORDENHAM, SEAHU.>.DEBRV, STADE])


['Destination']

In [6]:
df_final = pd.concat(dfs_clean, ignore_index=True)

In [7]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1060708 entries, 0 to 1060707
Data columns (total 20 columns):
 #   Column          Non-Null Count    Dtype              
---  ------          --------------    -----              
 0   TripID          1060708 non-null  int64              
 1   StartLatitude   1060708 non-null  float64            
 2   StartLongitude  1060708 non-null  float64            
 3   StartTime       1060708 non-null  datetime64[ns, UTC]
 4   EndLatitude     1060708 non-null  float64            
 5   EndLongitude    1060708 non-null  float64            
 6   EndTime         1060708 non-null  datetime64[ns, UTC]
 7   StartPort       1060708 non-null  string             
 8   EndPort         1060708 non-null  string             
 9   time            1060708 non-null  datetime64[ns, UTC]
 10  shiptype        1060708 non-null  int64              
 11  Length          1060708 non-null  int64              
 12  Breadth         1060708 non-null  int64              
 1

In [8]:
df_final = df_final.sort_values(['TripID', 'time']).reset_index(drop=True)
df_final.head()

,TripID,StartLatitude,StartLongitude,StartTime,EndLatitude,EndLongitude,EndTime,StartPort,EndPort,time,shiptype,Length,Breadth,Draught,Latitude,Longitude,SOG,COG,TH,Destination
0,5944,54.36,10.14,2016-08-08 04:10:00+00:00,54.52,18.52,2016-08-09 12:31:00+00:00,KIEL,GDYNIA,2016-08-08 04:10:00+00:00,70,82,12,5.3,54.36,10.14,0.7,90.0,106,GDYNIA.VIA.NOK
1,5944,54.36,10.14,2016-08-08 04:10:00+00:00,54.52,18.52,2016-08-09 12:31:00+00:00,KIEL,GDYNIA,2016-08-08 04:10:00+00:00,70,82,12,5.3,54.36,10.14,0.7,90.0,106,GDYNIA.VIA.NOK
2,5944,54.36,10.14,2016-08-08 04:10:00+00:00,54.52,18.52,2016-08-09 12:31:00+00:00,KIEL,GDYNIA,2016-08-08 04:10:00+00:00,70,82,12,5.3,54.36,10.14,0.7,90.0,106,GDYNIA.VIA.NOK
3,5944,54.36,10.14,2016-08-08 04:10:00+00:00,54.52,18.52,2016-08-09 12:31:00+00:00,KIEL,GDYNIA,2016-08-08 04:11:00+00:00,70,82,12,5.3,54.36,10.14,1.9,91.4,107,GDYNIA.VIA.NOK
4,5944,54.36,10.14,2016-08-08 04:10:00+00:00,54.52,18.52,2016-08-09 12:31:00+00:00,KIEL,GDYNIA,2016-08-08 04:11:00+00:00,70,82,12,5.3,54.36,10.14,1.9,91.4,107,GDYNIA.VIA.NOK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060703,2278147,53.58,8.52,2016-10-19 22:10:00+00:00,53.53,9.91,2016-10-20 06:49:00+00:00,BREMERHAVEN,HAMBURG,2016-10-20 06:45:00+00:00,71,150,23,8.3,53.53,9.91,0.4,313.9,297,HAMBURG
1060704,2278147,53.58,8.52,2016-10-19 22:10:00+00:00,53.53,9.91,2016-10-20 06:49:00+00:00,BREMERHAVEN,HAMBURG,2016-10-20 06:46:00+00:00,71,150,23,8.3,53.53,9.91,0.8,253.3,296,HAMBURG
1060705,2278147,53.58,8.52,2016-10-19 22:10:00+00:00,53.53,9.91,2016-10-20 06:49:00+00:00,BREMERHAVEN,HAMBURG,2016-10-20 06:47:00+00:00,71,150,23,8.3,53.53,9.91,0.8,266.5,306,HAMBURG
1060706,2278147,53.58,8.52,2016-10-19 22:10:00+00:00,53.53,9.91,2016-10-20 06:49:00+00:00,BREMERHAVEN,HAMBURG,2016-10-20 06:48:00+00:00,71,150,23,8.3,53.53,9.91,0.5,281.7,305,HAMBURG


In [9]:
df_final.to_parquet(output_file)